In [46]:
import pandas as pd
import netCDF4 as nc
import numpy as np
import xarray as xr

In [47]:
drifter_2005=nc.Dataset('NOAA_drifter_2005.nc','r')

In [48]:
times_2005=drifter_2005.variables['times'][:]
lons_2005=drifter_2005.variables['lons'][:]
lats_2005=drifter_2005.variables['lats'][:]
drifter_times = pd.DataFrame(times_2005)
drifter_lons = pd.DataFrame(lons_2005)
drifter_lats=pd.DataFrame(lats_2005)
drifter_times=drifter_times*24

In [49]:
for i in range(31):
    var=f"drifter{i+1}"
    globals()[var]=pd.DataFrame({'time':drifter_times.loc[i] , 'lons':drifter_lons.loc[i], 'lats' : drifter_lats.loc[i]}).dropna()

drifter_2005.close()

In [52]:
# .nc 파일 열기
dataset = xr.open_dataset('interpolated_sea.nc', engine='netcdf4')

for i in range(31):
    name=f"drifter{i+1}"
    if name in globals():
        drifter = globals()[name]
        uo_list=[]
        vo_list=[]
        for j in range(len(drifter)):
            input_time_hours=drifter.loc[j]['time']
            # 시간을 datetime 형식으로 변환0
            base_time = np.datetime64('2005-01-01T00:00:00')
            input_time = base_time + np.timedelta64(int(input_time_hours), 'h')
            input_longitude = drifter.loc[j]['lons']  # 경도 입력
            input_latitude = drifter.loc[j]['lats']   # 위도 입력

            nearest_point = dataset['uo'].sel(
            time=input_time,
            longitude=input_longitude,
            latitude=input_latitude,
            method='nearest'
            )
            uo_value = nearest_point.item()
            uo_list.append(uo_value)

            nearest_point = dataset['vo'].sel(
            time=input_time,
            longitude=input_longitude,
            latitude=input_latitude,
            method='nearest'
            )
            vo_value = nearest_point.item()
            vo_list.append(vo_value)

    drifter['uo']=uo_list
    drifter['vo']=vo_list

In [62]:
# .nc 파일 열기
dataset = xr.open_dataset('interpolated_wind.nc', engine='netcdf4')

for i in range(31):
    name=f"drifter{i+1}"
    if name in globals():
        drifter = globals()[name]
        u10_list=[]
        v10_list=[]
        for j in range(len(drifter)):
            input_time_hours=drifter.loc[j]['time']
            # 시간을 datetime 형식으로 변환0
            base_time = np.datetime64('2005-01-01T00:00:00')
            input_time = base_time + np.timedelta64(int(input_time_hours), 'h')
            input_longitude = drifter.loc[j]['lons']  # 경도 입력
            input_latitude = drifter.loc[j]['lats']   # 위도 입력

            nearest_point = dataset['u10'].sel(
            time=input_time,
            longitude=input_longitude,
            latitude=input_latitude,
            method='nearest'
            )
            u10_value = nearest_point.item()
            u10_list.append(u10_value)

            nearest_point = dataset['v10'].sel(
            time=input_time,
            longitude=input_longitude,
            latitude=input_latitude,
            method='nearest'
            )
            v10_value = nearest_point.item()
            v10_list.append(v10_value)

    drifter['u10']=u10_list
    drifter['v10']=v10_list

In [63]:
drifter5

,time,lons,lats,uo,vo,u10,v10
0,2244.0,129.332001,34.980000,0.307683,0.106753,7.613243,4.630820
1,2250.0,129.395004,34.987000,0.297363,0.076145,8.830918,3.419412
2,2256.0,129.429993,34.974998,0.311670,0.065315,8.847443,5.997562
3,2262.0,129.585999,35.036999,0.337904,0.122344,8.576849,8.340270
4,2268.0,129.727997,35.036999,0.326438,0.112608,7.777458,8.769147
...,...,...,...,...,...,...,...
420,4764.0,139.328995,38.341000,NaN,NaN,3.241384,4.442270
421,4770.0,139.386002,38.416000,NaN,NaN,7.576062,-0.614383
422,4776.0,139.464996,38.501999,0.176825,0.060967,3.537798,-1.787691
423,4782.0,139.503998,38.666000,NaN,NaN,4.722425,0.028445


In [57]:
# .nc 파일 열기
dataset = xr.open_dataset('interpolated_sea.nc', engine='netcdf4')  # 자료 파일 이름을 지정하세요

input_time_hours = float(input("Enter time (hours since 2005-01-01 00:00:00): "))  # 시간 입력 (시간 단위)

# 시간을 datetime 형식으로 변환
base_time = np.datetime64('2005-01-01T00:00:00')
input_time = base_time + np.timedelta64(int(input_time_hours), 'h')

# 사용자로부터 입력받은 경도와 위도
input_longitude = float(input("Enter longitude: "))  # 경도 입력
input_latitude = float(input("Enter latitude: "))    # 위도 입력

# 입력된 시간과 위치에 가장 가까운 자료 포인트 선택
nearest_point = dataset['uo'].sel(
    time=input_time,
    longitude=input_longitude,
    latitude=input_latitude,
    method='nearest'
)

uo_value = nearest_point.item()

nearest_point = dataset['vo'].sel(
    time=input_time,
    longitude=input_longitude,
    latitude=input_latitude,
    method='nearest'
)

vo_value = nearest_point.item()


# 찾은 가장 가까운 포인트의 시간, 경도, 위도
#nearest_time = nearest_point.time.item()
#nearest_longitude = nearest_point.longitude.item()
#nearest_latitude = nearest_point.latitude.item()


vo_value = nearest_point.item()

# 결과 출력

print(f'uo value at nearest point and time: {uo_value} m/s')
print(f'vo value at nearest point and time: {vo_value} m/s')

# 파일을 닫습니다.
dataset.close()

Enter time (hours since 2005-01-01 00:00:00):  4956
Enter longitude:  137.875
Enter latitude:  37.20


uo value at nearest point and time: nan m/s
vo value at nearest point and time: nan m/s
